In [42]:
!pip install langchain_community

In [43]:
# ReAct Agent with Error Feedback Loop using josstheboss/nltosql_ollama_3B_V1

import sqlite3
import os
import json
from huggingface_hub import InferenceClient
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool




In [44]:
from huggingface_hub import login

login(token)

In [57]:

# Initialize Hugging Face Inference Client
client = InferenceClient(model = "jossthebos/fine_tuned_ollama_3B_V1_Quantized",token=token)


# llm = HuggingFaceEndpoint(
#     repo_id="jossthebos/fine_tuned_ollama_3B_V1",
#     task="text-generation"
# )

# Setup SQLite connection
conn = sqlite3.connect("student_data.db")
cursor = conn.cursor()

# SQL Execution Tool
@tool
def run_sql_query(query: str) -> str:
    """Executes a SQL query on the student database and returns results as JSON or error."""
    try:
        cursor.execute(query)
        rows = cursor.fetchall()
        col_names = [desc[0] for desc in cursor.description]
        result = [dict(zip(col_names, row)) for row in rows]
        return json.dumps({"result": result}, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})




In [46]:
# Callback Prompt Template
callback_prompt = PromptTemplate(
    input_variables=["nl_input", "schema", "previous_query", "error_message"],
    template="""
You are a helpful SQL generator. Based on the natural language question, database schema, and past failed attempt (if any), fix or generate a correct SQL query.
Schema:
{schema}

Question: {nl_input}

Previous Attempt:
{previous_query}

Error:
{error_message}

Generate the corrected SQL query:
"""
)

# # Tool wrapper
# sql_tool = Tool(
#     name="SQLExecutor",
#     func=run_sql_query,
#     description="Executes SQL queries against the student database and returns the result in JSON."
# )



In [47]:
# Initialize agent
# agent = initialize_agent(
#     tools=[sql_tool],
#     llm=None,  # Remove LLM as we use the InferenceClient directly
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True
# )


# Function to run ReAct agent in loop until valid SQL
def generate_and_validate_sql(question: str, schema: str, max_attempts: int = 3):
    previous_query = ""
    error_message = ""

    for attempt in range(max_attempts):
        print(f"\nAttempt {attempt + 1}:")
        prompt = callback_prompt.format(
            nl_input=question,
            schema=schema,
            previous_query=previous_query,
            error_message=error_message
        )

        # Use the InferenceClient to generate SQL
        response = client.text_generation(prompt,max_new_tokens = 256,temperature = 0.1)
        generated_sql = response[0]['generated_text'].strip()


        # # Corrected call to `chat_completion`
        # response = client.text_generation(
        #     messages=[{"role": "user", "content": prompt}],
        #     max_tokens=256,
        #     temperature=0.1
        # )

        generated_sql = response.choices[0].message["content"].strip()
        print("Generated SQL:", generated_sql)
        print("Generated SQL:", generated_sql)

        result = run_sql_query(generated_sql)
        result_data = json.loads(result)

        if "error" in result_data:
            print("Error occurred:", result_data["error"])
            previous_query = generated_sql
            error_message = result_data["error"]
        else:
            print("Success! Result:", json.dumps(result_data, indent=2))
            return result_data

    print("All attempts failed. Please refine your question.")
    return {"error": "Max retries exceeded."}



In [48]:
!huggingface-cli whoami

jossthebos


In [62]:
# Sample usage
if __name__ == "__main__":
    sample_question = "Which school has the highest average number of credits?"
    schema_str = """
    studentid (bigint), studentnk (bigint), studentname (text), schoolid (integer),
    region (varchar), charter (varchar(500)), teachername (text), lp (varchar(4)),
    lp_start (date), credits (double precision), no_day_active (numeric), schoolname (varchar(500))

    """
    final_output = generate_and_validate_sql(sample_question, schema_str)
    print("\nFinal Output:", json.dumps(final_output, indent=2))


Attempt 1:


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://router.huggingface.co/hf-inference/models/jossthebos/fine_tuned_ollama_3B_V1_Quantized (Request ID: Root=1-6800f940-3b2daa074babf4674c1ced9b;d1cac62a-eb76-4dca-9e50-77797f9f4ff5)

No module named 'optimum'

In [55]:
import huggingface_hub
print(huggingface_hub.__version__)


0.30.2


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
model_info = api.model_info("jossthebos/fine_tuned_ollama_3B_V1_Quantized")
print(model_info.pipeline_tag)


RuntimeError: GPU is required to quantize or run quantize model.